# RNA velocity

Instructions on how to generate the `.loom` files can be found in the documentation of [velocyto](http://velocyto.org/). Another option to do this is using [alevin](https://combine-lab.github.io/alevin-tutorial/2020/alevin-velocity/). We will analyze the data of Glutamaergic differentiation from the original publication of [RNA velocity](https://www.nature.com/articles/s41586-018-0414-6).

## Settings

In [ ]:
import scvelo as scv
import loompy as lp
import pandas as pd
import numpy as np
import umap
import scanpy
from matplotlib import pyplot as plt
import pickle
from matplotlib import colors

In [ ]:
scv.logging.print_version()
scv.settings.verbosity = 0  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.set_figure_params('scvelo')  # for beautified visualization

## Preprocessing

In [ ]:
adata = scv.read("../data/loom/a4.hgForebrainGlut.loom", cache=True)
adata.var_names_make_unique()

In [ ]:
scv.pl.proportions(adata)

In [ ]:
scv.pp.filter_and_normalize(adata,min_shared_counts=10,min_shared_cells=50,n_top_genes=2000)
scv.pp.neighbors(adata, n_pcs=20, n_neighbors=200)
scv.pp.moments(adata, n_pcs=20, n_neighbors=200)

## Dimensionality reduction and embbedding

In [ ]:
scanpy.tl.leiden(adata,0.5)
scv.tl.umap(adata)

## Estimations

### RNA velocity

In [ ]:
scv.tl.recover_dynamics(adata,n_top_genes=2000)
scv.tl.velocity(adata, mode='dynamical')

### Latent time

In [ ]:
scv.tl.recover_latent_time(adata)
scv.tl.terminal_states(adata)

### Downstream analysis

In [ ]:
scv.tl.velocity_confidence(adata)
scv.tl.score_genes_cell_cycle(adata)
scv.tl.rank_velocity_genes(adata, groupby='leiden',n_genes=500)

## Visualizations

**Velocity embeddings**

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color='leiden',legend_loc='right',n_neighbors=200,recompute=True,alpha=0.2)

In [ ]:
scv.pl.velocity_embedding(adata, arrow_length=3, arrow_size=2, dpi=120,color='leiden',legend_loc='right')

**Velocity confidence**

In [ ]:
scv.tl.velocity_confidence(adata)
tags = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(adata, c=tags, cmap='coolwarm', perc=[2, 98])

**Cell cycle scores**

In [ ]:
scv.pl.scatter(adata, color_gradients=['S_score', 'G2M_score'], smooth=True, perc=[5, 95])

**Pseudotime analysis**

In [ ]:
scv.pl.velocity_graph(adata, threshold=.1)
x, y = scv.utils.get_cell_transitions(adata, basis='umap', starting_cell=70)
ax = scv.pl.velocity_graph(adata, c='lightgrey', edge_width=.05, show=False)
ax = scv.pl.scatter(adata, x=x, y=y, s=120, c='ascending', cmap='gnuplot', ax=ax)

**Velocity graph**

In [ ]:
scv.tl.velocity_pseudotime(adata)
scv.pl.scatter(adata, color='velocity_pseudotime', cmap='gnuplot')

### Trajectory inference

In [ ]:
adata.uns['neighbors']['distances'] = adata.obsp['distances']
adata.uns['neighbors']['connectivities'] = adata.obsp['connectivities']

scv.tl.paga(adata, groups='leiden')
df = scv.get_df(adata, 'paga/transitions_confidence', precision=2).T
df.style.background_gradient(cmap='Blues').format('{:.2g}')

In [ ]:
scv.pl.paga(adata, basis='umap', size=50, alpha=.1, min_edge_width=2, node_size_scale=1.5)

### Hands-on activity 3

---

1. How many genes have valid estimates of RNA velocity?
2. What happens if you change the model to `deterministic`?